In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
%time df = pd.read_feather('tmp/raw')
df.head()

CPU times: total: 109 ms
Wall time: 301 ms


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,521D,...,16,3,320,False,False,False,False,False,False,1.163635e+09
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,950FII,...,26,4,86,False,False,False,False,False,False,1.080259e+09
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,226,...,26,3,57,False,False,False,False,False,False,1.077754e+09
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,19,3,139,False,False,False,False,False,False,1.305763e+09
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,23,3,204,False,False,False,False,False,False,1.248307e+09


## Step 0) What is the purpose of the project ?

* *The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuaration.*
* The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.

In [3]:
df.SalePrice = np.log(df.SalePrice)

## Step 1) Convert everything into numbers

In [4]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [5]:
def train_cats(df):
    for name,col in df.items():
        if not is_numeric_dtype(col): df[name] = df[name].astype('category').cat.as_ordered()

In [6]:
%time train_cats(df)

CPU times: total: 156 ms
Wall time: 409 ms


In [7]:
df.UsageBand.dtype

CategoricalDtype(categories=['High', 'Low', 'Medium'], ordered=True, categories_dtype=object)

In [8]:
df.UsageBand = df.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True)

## Step 2) Handle Missing Values

In [9]:
(df.isnull().sum()/len(df)).sort_values(ascending=False)

Pushblock                   0.937129
Tip_Control                 0.937129
Engine_Horsepower           0.937129
Enclosure_Type              0.937129
Blade_Width                 0.937129
Blade_Extension             0.937129
Scarifier                   0.937102
Hydraulics_Flow             0.891899
Grouser_Tracks              0.891899
Coupler_System              0.891660
fiModelSeries               0.858129
Steering_Controls           0.827064
Differential_Type           0.826959
UsageBand                   0.826391
fiModelDescriptor           0.820707
Backhoe_Mounting            0.803872
Turbocharged                0.802720
Pad_Type                    0.802720
Stick                       0.802720
Blade_Type                  0.800977
Travel_Controls             0.800975
Tire_Size                   0.763869
Track_Type                  0.752813
Grouser_Type                0.752813
Pattern_Changer             0.752651
Stick_Length                0.752651
Thumb                       0.752476
U

In [10]:
# see missing values
for name, col in df.items():
    if is_numeric_dtype(col): 
        if pd.isnull(col).sum(): 
            print(name, ": ", df[name].isnull().sum() / len(df))

auctioneerID :  0.050198815830476785
MachineHoursCurrentMeter :  0.6440885010906825


In [11]:
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name+"_na"] = pd.isnull(col)
        df[name] = col.fillna(col.median())

In [12]:
for name, col in df.items(): 
    fix_missing(df, col, name)

In [13]:
# checking
for name, col in df.items():
    if is_numeric_dtype(col): 
        print(name, ": ", df[name].isnull().sum() / len(df))

SalesID :  0.0
SalePrice :  0.0
MachineID :  0.0
ModelID :  0.0
datasource :  0.0
auctioneerID :  0.0
YearMade :  0.0
MachineHoursCurrentMeter :  0.0
saleYear :  0.0
saleMonth :  0.0
saleWeek :  0.0
saleDay :  0.0
saleDayofweek :  0.0
saleDayofyear :  0.0
saleIs_month_end :  0.0
saleIs_month_start :  0.0
saleIs_quarter_end :  0.0
saleIs_quarter_start :  0.0
saleIs_year_end :  0.0
saleIs_year_start :  0.0
saleElapsed :  0.0
auctioneerID_na :  0.0
MachineHoursCurrentMeter_na :  0.0


## Step 3) Encoding
 

### Label encoding

In [14]:
def numericalize(df):
    for name, col in df.items():
        if not is_numeric_dtype(col) and col.nunique() > 0: 
            df[name] = 1 + df[name].cat.codes

In [15]:
numericalize(df)

### One-Hot-Encoding

In [16]:
cats_cols = []
for name, col in df.items():
    if not is_numeric_dtype(col): cats_cols.append(name)

In [17]:
len(cats_cols)

0

In [18]:
df = pd.get_dummies(df, columns=cats_cols, dummy_na=True)
df.shape

(401125, 67)

## Save processed data

In [19]:
%time df.to_feather('tmp/raw_processed')

CPU times: total: 46.9 ms
Wall time: 43.8 ms
